# Example

This example goes through the main usages of gizio. If you'd like to follow along, make sure gizio is [installed](about.rst#installation), and the [sample data](http://yt-project.org/data/FIRE_M12i_ref11.tar.gz) (from [yt-project](http://yt-project.org/)) is downloaded.

In [ ]:
import gizio

## Snapshot

### Loading

A snapshot can be loaded by its path:

In [ ]:
snap = gizio.Snapshot('data/FIRE_M12i_ref11/snapshot_600.hdf5')

Or its prefix (including the directory part):

In [ ]:
snap = gizio.Snapshot('data/FIRE_M12i_ref11/snapshot_600')

Or its directory (if it's the only snapshot in that directory):

In [ ]:
snap = gizio.Snapshot('data/FIRE_M12i_ref11')

The latter two forms are useful when loading multi-file snapshot.

### Inspection

After a snapshot is loaded, its content could be inspected from several attributes:

In [ ]:
snap.header

In [ ]:
snap.shape

In [ ]:
snap.keys

### Field Access

For any available key listed above, the field could be accessed through a dictionary interface:

In [ ]:
snap['PartType0', 'Coordinates']

Note that the loaded array has units, which is supported by [unyt](https://unyt.readthedocs.io).

## Particle Selector

Particle selector is a more flexible interface to access fields. Default particle selectors according to particle types are created automatically, and could be accessed from `snap.pt`:

In [ ]:
snap.pt

Let's have a look at the gas particles:

In [ ]:
gas = snap.pt['gas']
gas.keys()

Different from the snapshot interface, shorthand keys are used in the particle selector interface, except for unrecognized ones like `'Potential'`. So we have the following equality:

In [ ]:
(gas['p'] == snap['PartType0', 'Coordinates']).all()

The number of selected particles can be gotten from `len`:

In [ ]:
len(gas)

### Derived Fields

Among the listed keys, one is special: `'t'` for temperature. There isn't a `('PartType0', 'Temperature')` field in the snapshot. Actually, temperature is computed from internal energy. So we call it a derived field:

In [ ]:
gas['t']

We could check the code that does the computation (be careful this is not part of the public interface yet):

In [ ]:
import inspect
print(inspect.getsource(gas._func_registry['t']))

It's also pretty easy to register a custom derived field:

In [ ]:
def volume(field_system):
    return (field_system['m'] / field_system['rho']).to('kpc**3')

gas.register('vol', volume)
gas['vol']

### Boolean Masking

The particle selection could be further refined by boolean masking. For example, to select hot gas, we could do:

In [ ]:
hot_gas = gas[gas['t'].to_value('K') > 1e6]
hot_gas['t'].min()

### Set-like Operations

Another way to construct a new particle selector is to combine existing ones. A set-like interface is implemented for particle selectors to combine. For example, to define baryon as gas or star:

In [ ]:
star = snap.pt['star']
baryon = gas | star

Only keys from both gas and star are kept in the result:

In [ ]:
set(baryon.keys()) == set(gas.keys()) & set(star.keys())

Here's another example to define dark matter as all but baryon:

In [ ]:
dm = snap.pt['all'] - baryon